In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

### embedding

In [2]:
word = {'hello':0 , 'world':1}

In [3]:
a = torch.tensor([word['hello']])

In [4]:
embed= nn.Embedding(2,5)

In [5]:
embed(a)

tensor([[ 0.1163,  0.0230,  0.0517, -0.7425,  0.7532]],
       grad_fn=<EmbeddingBackward>)

### 压缩f 的维度从5到1

In [6]:
data = torch.rand(10 , 48  , 5) #  v ,t , f 

In [7]:
data = data.view(10, -1)    #  v , t  

In [8]:
data.shape

torch.Size([10, 240])

In [9]:
linear1 = nn.Linear(240 , 48)
output = linear1(data)
output.reshape(10,48 ,1)

tensor([[[-0.3243],
         [ 0.4059],
         [-0.0416],
         [-0.2535],
         [ 0.2434],
         [-0.1275],
         [-0.0344],
         [-0.0916],
         [-0.1420],
         [ 0.3764],
         [ 0.3226],
         [ 0.2982],
         [-0.2660],
         [ 0.1123],
         [-0.5086],
         [-0.0021],
         [-0.1456],
         [-0.0865],
         [-0.5184],
         [ 0.3233],
         [-0.0279],
         [-0.6809],
         [-0.5108],
         [-0.5079],
         [ 0.2120],
         [ 0.0464],
         [-0.5996],
         [ 0.7858],
         [-0.4039],
         [ 0.0581],
         [-0.0894],
         [-0.2846],
         [ 0.2320],
         [-0.3409],
         [-0.5394],
         [-0.3457],
         [ 0.1426],
         [-0.2314],
         [-0.4082],
         [-0.2788],
         [ 0.1929],
         [ 0.1521],
         [ 0.1015],
         [-0.3481],
         [ 0.0471],
         [-0.2082],
         [-0.5511],
         [-0.0266]],

        [[-0.1885],
         [ 0.2708]

### 参数标准化 uniform_

In [10]:
from torch.nn.parameter import Parameter
p1 = Parameter(torch.FloatTensor(4, 4))
p1

Parameter containing:
tensor([[7.8473e-44, 8.1275e-44, 6.7262e-44, 7.8473e-44],
        [8.1275e-44, 7.2868e-44, 7.5670e-44, 6.4460e-44],
        [6.7262e-44, 7.7071e-44, 7.8473e-44, 7.7071e-44],
        [7.0065e-44, 7.0065e-44, 1.2612e-43, 0.0000e+00]], requires_grad=True)

In [11]:
p1.data.uniform_(-0.5 , 0.5)

tensor([[-0.4916, -0.4412, -0.0049, -0.2021],
        [ 0.3667,  0.0841,  0.1413, -0.3793],
        [-0.2038, -0.1635,  0.3213,  0.3987],
        [ 0.2851, -0.3253, -0.4595, -0.2169]])

In [14]:
import math
weight = Parameter(torch.FloatTensor(4, 4))
stdv = 1. / math.sqrt(weight.size(1))
weight.data.uniform_(-stdv, stdv)

tensor([[ 0.2734,  0.0336,  0.0456, -0.0205],
        [ 0.3176,  0.3502,  0.4384,  0.4363],
        [ 0.4543,  0.0495, -0.2174, -0.2109],
        [-0.4292, -0.4744,  0.4126, -0.0116]])

### 定义参数

In [15]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.param = nn.Parameter(torch.rand(3,3)) 
        # 等价与self.register_parameter('param1' ,nn.Parameter(t.randn(3, 3)))
    def forward(self,input):
        x = self.param.mm(input)
        return x 
n = Net()   

In [16]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.register_parameter('param',nn.Parameter(torch.randn(3, 3)))
    def forward(self,input):
        x = self.param.mm(input)
        return x 
n = Net()   

### cluster gcn

In [17]:
c = torch.randint(3, 5, (10,2))  # n , k
w = torch.randint(3, 5, (2,3,4)) #  k , f1, f2 

In [18]:
out = torch.mm(c,w.view(2,-1)) # n , f1*f2

In [19]:
out.view(10,3,4).shape # n , f1 , f2 

torch.Size([10, 3, 4])

In [20]:
W = nn.Parameter(torch.rand(10,2,3)) 

In [21]:
con1 = nn.Conv2d(1, 1, 16) 

In [22]:
def cw( c , w):
    """
    input:  
    output : w of clustered 
    """
    CW = []
    for i in range(c.shape[0]):
        w_i = c[i][0] * w[0] + c[i][1]*w[1] + c[i][2] *w[2] 
        CW.append(w_i)
    return CW 

In [23]:
c = torch.rand(10 ,3 )  # v , k 
w = torch.rand(3,2,2)  # v , f1, f2

### soft cluster

In [24]:
linear1 = nn.Linear(5 , 1) 

In [25]:
x = torch.randn(4,2, 3, 5)

In [26]:
linear1(x).view(4,2,3).shape

torch.Size([4, 2, 3])

In [27]:
softmax = nn.Softmax(dim=-1)

In [28]:
softmax(x).shape

torch.Size([4, 2, 3, 5])

### 矩阵间的欧式距离


In [29]:
A = torch.Tensor( [[0.1 , 0.9], [0.2,0.8]] ) 
B = torch.Tensor([[0.2 , 0.8], [0.1,0.9] ])
C = torch.Tensor([[0.1 , 0.9], [0.2,0.8] ])
C = A - B 
C

tensor([[-0.1000,  0.1000],
        [ 0.1000, -0.1000]])

In [30]:
torch.sum(C**2)

tensor(0.0400)

In [31]:
def cluster_distance(A,B):
    D = A - B
    return torch.sum( D**2) 

In [32]:
# 排列组合
import itertools
list(itertools.permutations([1,2,3,4],2))
list(itertools.combinations([1,2,3,4],2))

[(1, 2), (1, 3), (1, 4), (2, 3), (2, 4), (3, 4)]

In [33]:
A = torch.Tensor( [[0.1 , 0.9], [0.2,0.8]] ) 
B = torch.Tensor([[0.2 , 0.8], [0.1,0.9] ])
C = torch.Tensor([[0.1 , 0.9], [0.2,0.8] ])
D = torch.Tensor([[0.3 , 0.7], [0.6,0.4] ])
list(itertools.combinations([A , B , C],2))

[(tensor([[0.1000, 0.9000],
          [0.2000, 0.8000]]), tensor([[0.2000, 0.8000],
          [0.1000, 0.9000]])), (tensor([[0.1000, 0.9000],
          [0.2000, 0.8000]]), tensor([[0.1000, 0.9000],
          [0.2000, 0.8000]])), (tensor([[0.2000, 0.8000],
          [0.1000, 0.9000]]), tensor([[0.1000, 0.9000],
          [0.2000, 0.8000]]))]

In [34]:
def matrix_distance(A,B):
    D = A - B
    return torch.sum(D**2) 

In [35]:
import itertools
def cluster_distance(m_list):
    p = list(itertools.combinations(m_list,2))
    res = 0 
    for item in p:
        res_item = matrix_distance(item[0] , item[1])
        res += res_item
    return res

In [36]:
cluster_distance([A,B,C,D])

tensor(1.4000)

### dataloader 

In [3]:
import torchvision
import torchvision.transforms as transforms
train_dataset = torch.rand(50, 10 , 5)

In [4]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=10, 
                                           shuffle=True)

In [5]:
len(train_loader)

5

In [6]:
for i, data in enumerate(train_loader): 
    print(i,data.size())

0 torch.Size([10, 10, 5])
1 torch.Size([10, 10, 5])
2 torch.Size([10, 10, 5])
3 torch.Size([10, 10, 5])
4 torch.Size([10, 10, 5])


### cluster distance 

In [7]:
import itertools
def matrix_distance(A,B):
    D = A - B
    return torch.mean(D**2) 
def cluster_distance(m_list):
    p = list(itertools.combinations(m_list,2))
    res = []
    for item in p:
        res_item = matrix_distance(item[0] , item[1])
        res.append(res_item)
        res = torch.Tensor( res ) 
    return torch.mean(res)

In [8]:
A = torch.Tensor( [[0.1 , 0.9], [0.2,0.8]] ) 
B = torch.Tensor([[0.2 , 0.8], [0.1,0.9] ])

In [9]:
cluster_distance([A,B])

tensor(0.0100)

### graph to batch graph

In [10]:
graph = torch.Tensor( [ [2,3], [5,6] ])
graph.unsqueeze(0)
graph = graph.repeat(2 ,1,1) 
print(graph)

tensor([[[2., 3.],
         [5., 6.]],

        [[2., 3.],
         [5., 6.]]])


### 软分类 to 硬分类

In [11]:
cluster = torch.Tensor([[0.9,0.1],[0.2,0.8] , [0.8,0.2] , [0.7,0.3] ])
cluster

tensor([[0.9000, 0.1000],
        [0.2000, 0.8000],
        [0.8000, 0.2000],
        [0.7000, 0.3000]])

In [12]:
torch.max(cluster , 1)[0]

tensor([0.9000, 0.8000, 0.8000, 0.7000])

In [13]:
m = torch.transpose(torch.max(cluster , 1)[0].repeat(2,1),0,1)
a = torch.ones(4,2)
b = torch.zeros(4,2)
torch.where(cluster-m>=0 , a, b)

tensor([[1., 0.],
        [0., 1.],
        [1., 0.],
        [1., 0.]])

### 加batch

In [14]:
soft_cluster = torch.Tensor([[[0.9,0.1,0.1],[0.2,0.4,0.8] , [0.8,0.6,0.2] , [0.7,0.3,0.1]]])
soft_cluster.shape

torch.Size([1, 4, 3])

In [15]:
max_mat = torch.max(soft_cluster, -1)[0].unsqueeze(-1).repeat(1, 1, 3)
max_mat

tensor([[[0.9000, 0.9000, 0.9000],
         [0.8000, 0.8000, 0.8000],
         [0.8000, 0.8000, 0.8000],
         [0.7000, 0.7000, 0.7000]]])

In [16]:
out = torch.rand([1, 4 ,3])

In [17]:
torch.max(soft_cluster, -1)[0]

tensor([[0.9000, 0.8000, 0.8000, 0.7000]])

In [18]:
a = torch.ones(1,4,3)
b = torch.zeros(1,4,3)

In [19]:
hard_cluster = torch.where(soft_cluster-max_mat >=0 , a , b)
hard_cluster

tensor([[[1., 0., 0.],
         [0., 0., 1.],
         [1., 0., 0.],
         [1., 0., 0.]]])

In [20]:
hard_cluster[:, :, 0]

tensor([[1., 0., 1., 1.]])

In [21]:
out

tensor([[[0.8020, 0.2433, 0.5654],
         [0.7393, 0.5971, 0.2951],
         [0.7850, 0.3613, 0.4925],
         [0.8192, 0.9343, 0.2774]]])

In [22]:
torch.mul(out ,hard_cluster[:, :, 0].unsqueeze(-1))

tensor([[[0.8020, 0.2433, 0.5654],
         [0.0000, 0.0000, 0.0000],
         [0.7850, 0.3613, 0.4925],
         [0.8192, 0.9343, 0.2774]]])

### 构建sparse matrix

In [23]:
import numpy as np

In [24]:
graph_2D = torch.FloatTensor((np.random.rand(4,4)>0.9).astype(np.int))
graph_2D

tensor([[0., 1., 0., 1.],
        [0., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 0., 0., 0.]])

### 转置

In [25]:
x = torch.Tensor([[1,2],[3,4]])
x

tensor([[1., 2.],
        [3., 4.]])

In [26]:
torch.transpose(x, 0, 1)

tensor([[1., 3.],
        [2., 4.]])

In [27]:
x.t()

tensor([[1., 3.],
        [2., 4.]])

In [28]:
a = torch.rand(1,1,2)

In [29]:
a.squeeze(0).shape

torch.Size([1, 2])

In [30]:
a.squeeze().shape

torch.Size([2])

In [31]:
a = torch.rand(3,3, 5)
b = torch.rand(5,1)
torch.matmul(a,b).shape

torch.Size([3, 3, 1])